In [ ]:
from __future__ import division
from datetime import datetime, timedelta,date
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")

import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.layers import LSTM
from sklearn.model_selection import KFold, cross_val_score, train_test_split
pyoff.init_notebook_mode()


In [3]:
df_sales = pd.read_csv('lstm1.csv')
df_sales.head()


,arrival_date,sales
0,01/01/2019,406.0008
1,01/01/2019,826.9992
2,01/01/2019,52.7082
3,01/01/2019,101.5002
4,01/01/2019,203.0004


In [4]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114594 entries, 0 to 114593
Data columns (total 2 columns):
arrival_date    114594 non-null object
sales           114594 non-null float64
dtypes: float64(1), object(1)
memory usage: 1.7+ MB


In [6]:
df_sales['arrival_date'] = pd.to_datetime(df_sales['arrival_date'])
df_sales.set_index('arrival_date', inplace=True)
#df_sales['arrival_date'] = pd.to_datetime(df_sales['arrival_date'])

In [7]:

#groupby date and sum the sales
#df_sales = df_sales.groupby('arrival_date').sales.sum().reset_index()
#df_sales['arrival_date'] = pd.to_datetime(df_sales['arrival_date'],unit='w')
#df_sales.set_index('arrival_date', inplace=True)
#week_dff = df_sales.groupby(df_sales['arrival_date'].dt.weekday_name).mean()
#print (week_dff)
#df_sales.index = pd.to_datetime(df_sales['date'], unit='d')
week_df = df_sales.resample('W').sum()
#df_sales['date'] = pd.to_datetime(df_sales['date'])

In [8]:
week_df.head()

,sales
arrival_date,
2019-01-06,4.616801e+06
2019-01-13,5.132808e+06
2019-01-20,8.818287e+06
2019-01-27,6.468251e+06
2019-02-03,1.022050e+07


In [ ]:
#df_sales = df_sales.groupby('arrival_date').sales.sum().reset_index()

In [ ]:
#df_sales.head()


In [ ]:
#df_sales.resample('W').sum()
#df_sales['arrival_date'] = pd.to_datetime(df_sales['arrival_date'], unit='d')
# week_df = df_sales.resample('W').mean()
# print(week_df)
#df_sales.head()


In [ ]:
# df_sales.drop(['arrival_date'],axis=1)

In [ ]:

#df_sales.info()

In [ ]:
# week_df.head(10)

In [ ]:
#represent month in date field as its first day
#df_sales['date'] = df_sales['date'].dt.year.astype('str') + '-' + df_sales['date'].dt.month.astype('str') + '-01'
#df_sales['date'] = pd.to_datetime(df_sales['date'])
#groupby date and sum the sales
#df_sales = df_sales.groupby('date').sales.sum().reset_index()

In [ ]:
#df_sales.head()

In [9]:
#plot monthly sales
plot_data = [
    go.Scatter(
        x=week_df['arrival_date'],
        y=week_df['sales'],
    )
]
plot_layout = go.Layout(
        title='Daily Sales'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

KeyError: 'arrival_date'

In [ ]:
# #plot monthly sales
# plot_data = [
#     go.Scatter(
#         x=week_df['arrival_date'],
#         y=week_df['sales'],
#     )
# ]
# plot_layout = go.Layout(
#         title='Monthly Sales'
#     )
# fig = go.Figure(data=plot_data, layout=plot_layout)
# pyoff.iplot(fig)

In [ ]:
#df_sales.index = pd.to_datetime(df_sales['arrival_date'], unit='d')

In [ ]:
# df_sales.drop(['arrival_date'],axis=1)
# df_sales.head()

In [ ]:
# df_sales.drop(['arrival_date'],axis=1,inplace=True)
# df_sales.head()

In [10]:
#create a new dataframe to model the difference
df_diff = week_df.copy()
#add previous sales to the next row
df_diff['prev_sales'] = df_diff['sales'].shift(1)
#drop the null values and calculate the difference
df_diff = df_diff.dropna()
df_diff['diff'] = (df_diff['sales'] - df_diff['prev_sales'])

In [11]:
df_diff.head(10)

,sales,prev_sales,diff
arrival_date,,,
2019-01-13,5.132808e+06,4.616801e+06,5.160073e+05
2019-01-20,8.818287e+06,5.132808e+06,3.685479e+06
2019-01-27,6.468251e+06,8.818287e+06,-2.350036e+06
2019-02-03,1.022050e+07,6.468251e+06,3.752253e+06
2019-02-10,1.590785e+07,1.022050e+07,5.687349e+06
2019-02-17,8.121274e+06,1.590785e+07,-7.786578e+06
2019-02-24,1.346009e+07,8.121274e+06,5.338815e+06
2019-03-03,3.863888e+06,1.346009e+07,-9.596202e+06
2019-03-10,8.853227e+06,3.863888e+06,4.989338e+06


In [12]:
df_diff.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52 entries, 2019-01-13 to 2020-01-05
Freq: W-SUN
Data columns (total 3 columns):
sales         52 non-null float64
prev_sales    52 non-null float64
diff          52 non-null float64
dtypes: float64(3)
memory usage: 1.6 KB


In [13]:
#create dataframe for transformation from time series to supervised
df_supervised = df_diff.drop(['prev_sales'],axis=1)
#adding lags
for inc in range(1):
    field_name = 'lag_' + str(inc)
    df_supervised[field_name] = df_supervised['diff'].shift(inc)
#drop null values
df_supervised = df_supervised.dropna().reset_index(drop=False)

In [14]:
df_supervised.head()

,arrival_date,sales,diff,lag_0
0,2019-01-13,5.132808e+06,5.160073e+05,5.160073e+05
1,2019-01-20,8.818287e+06,3.685479e+06,3.685479e+06
2,2019-01-27,6.468251e+06,-2.350036e+06,-2.350036e+06
3,2019-02-03,1.022050e+07,3.752253e+06,3.752253e+06
4,2019-02-10,1.590785e+07,5.687349e+06,5.687349e+06


In [15]:
#plot sales diff
plot_data = [
    go.Scatter(
        x=df_diff['arrival_date'],
        y=df_diff['diff'],
    )
]
plot_layout = go.Layout(
        title='Montly Sales Diff'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

KeyError: 'arrival_date'

In [34]:
#create dataframe for transformation from time series to supervised
df_supervised = df_diff.drop(['prev_sales'],axis=1)
#adding lags
for inc in range(1,4):
    field_name = 'lag_' + str(inc)
    df_supervised[field_name] = df_supervised['diff'].shift(inc)
#drop null values
#df_supervised = df_supervised.dropna().reset_index(drop=False)

In [35]:
df_supervised.head()

,sales,diff,lag_1,lag_2,lag_3
arrival_date,,,,,
2019-01-13,5.132808e+06,5.160073e+05,NaN,NaN,NaN
2019-01-20,8.818287e+06,3.685479e+06,5.160073e+05,NaN,NaN
2019-01-27,6.468251e+06,-2.350036e+06,3.685479e+06,5.160073e+05,NaN
2019-02-03,1.022050e+07,3.752253e+06,-2.350036e+06,3.685479e+06,5.160073e+05
2019-02-10,1.590785e+07,5.687349e+06,3.752253e+06,-2.350036e+06,3.685479e+06


In [36]:
import statsmodels.formula.api as smf
# Define the regression formula
model = smf.ols(formula='diff ~ lag_1', data=df_supervised)
# Fit the regression
model_fit = model.fit()
# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

0.05930610244738599


In [19]:
from sklearn.preprocessing import MinMaxScaler

In [20]:
df_model = df_supervised.drop(['sales','arrival_date'],axis=1)
#split train and test set
train_set, test_set = df_model[0:-6].values, df_model[-6:].values
df_model.info()

KeyError: "['arrival_date'] not found in axis"

In [ ]:
#apply Min Max Scaler
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(train_set)
# reshape training set
train_set = train_set.reshape(train_set.shape[0], train_set.shape[1])
train_set_scaled = scaler.transform(train_set)
# reshape test set
test_set = test_set.reshape(test_set.shape[0], test_set.shape[1])
test_set_scaled = scaler.transform(test_set)

In [ ]:
X_train, y_train = train_set_scaled[:, 1:], train_set_scaled[:, 0:1]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0:1]
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [ ]:
# model = Sequential()
# model.add(LSTM(4, batch_input_shape=(3, X_train.shape[1], X_train.shape[2]), stateful=True))
# model.add(Dense(15, input_dim=10, activation = 'relu'))
# model.add(Dense(12, activation = 'relu'))
# model.add(Dense(10, activation = 'relu'))
# model.add(Dense(8, activation = 'relu'))
# model.add(Dense(1, activation = 'sigmoid'))
# model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])
# model.fit(X_train, y_train, nb_epoch=50, batch_size=5, shuffle=False)

#model = Sequential()
#model.add(LSTM(4, batch_input_shape=(4, X_train.shape[1], X_train.shape[2]), stateful=True))
#model.add(Dense(15,activation="relu"),(12,activation="relu"),Dense(8,activation="relu"),Dense(5,activation="sigmoid"))
#model.compile(loss='mean_squared_error', optimizer='adam')
#model.fit(X_train, y_train, nb_epoch=10, batch_size=32, shuffle=False)

model = Sequential()
model.add(LSTM(4, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, nb_epoch=10, batch_size=1, verbose=1, shuffle=False)

In [ ]:
y_pred = model.predict(X_test,batch_size=1)

In [ ]:
y_pred


In [ ]:
y_test


In [ ]:
y_pred = y_pred.reshape(y_pred.shape[0], 1, y_pred.shape[1])

In [ ]:
#rebuild test set for inverse transform
pred_test_set = []
for index in range(0,len(y_pred)):
    print (np.concatenate([y_pred[index],X_test[index]],axis=1,))
    pred_test_set.append(np.concatenate([y_pred[index],X_test[index]],axis=1))

In [ ]:
pred_test_set[0]

In [ ]:
pred_test_set = np.array(pred_test_set)
pred_test_set = pred_test_set.reshape(pred_test_set.shape[0], pred_test_set.shape[2])

In [ ]:
pred_test_set_inverted = scaler.inverse_transform(pred_test_set)

In [ ]:
#create dataframe that shows the predicted sales
result_list = []
sales_dates = list(df_sales[-7:].arrival_date)
act_sales = list(df_sales[-7:].sales)
for index in range(0,len(pred_test_set_inverted)):
    result_dict = {}
    result_dict['pred_value'] = int(pred_test_set_inverted[index][0] + act_sales[index])
    result_dict['arrival_date'] = sales_dates[index+1]
    result_list.append(result_dict)
df_result = pd.DataFrame(result_list)

In [ ]:
df_result

In [ ]:
#merge with actual sales dataframe
df_sales_pred = pd.merge(df_sales,df_result,on='arrival_date',how='left')

In [ ]:
df_sales_pred

In [ ]:
df_sales_pred.tail(20)

In [ ]:
plot_data = [
    go.Scatter(
        x=df_sales_pred['arrival_date'],
        y=df_sales_pred['sales'],
        name='actual'
    ),
        go.Scatter(
        x=df_sales_pred['arrival_date'],
        y=df_sales_pred['pred_value'],
        name='predicted'
    )
    
]

plot_layout = go.Layout(
        title='Sales Prediction'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)